# 4. TELEMETRY & TIMESTAMPS

We will integrate our progress so far by visualizing the timestamp points on the track where team radio conversations take place for each driver, alongside analyzing the car's telemetry data at these specific locations.

### 4.1 CONSTRUCT DICTIONARY

In [ ]:
import fastf1
import os
import pickle
import numpy as np
import pandas as pd

parent_dir = os.path.dirname(os.path.realpath("4. Telemetry & Timestamps.ipynb")) 
data_path = os.path.join(parent_dir, 'Data Objects')

In [2]:
with open(os.path.join(data_path, 'team_radio23.pkl'), 'rb') as f:
    team_radio = pickle.load(f)
    
print(len(team_radio.keys()),"races in 2023!")

22 races in 2023!


In [3]:
with open(os.path.join(data_path, 'team_radio22.pkl'), 'rb') as f:
    team_radio22 = pickle.load(f)
    
print(len(team_radio22.keys()),"races in 2022!")

22 races in 2022!


In [6]:
team_radio.keys()

dict_keys(['Bahrain_Grand_Prix', 'Saudi_Arabian_Grand_Prix', 'Australian_Grand_Prix', 'Azerbaijan_Grand_Prix', 'Miami_Grand_Prix', 'Monaco_Grand_Prix', 'Spanish_Grand_Prix', 'Canadian_Grand_Prix', 'Austrian_Grand_Prix', 'British_Grand_Prix', 'Hungarian_Grand_Prix', 'Belgian_Grand_Prix', 'Dutch_Grand_Prix', 'Italian_Grand_Prix', 'Singapore_Grand_Prix', 'Japanese_Grand_Prix', 'Qatar_Grand_Prix', 'United_States_Grand_Prix', 'Mexico_City_Grand_Prix', 'São_Paulo_Grand_Prix', 'Las_Vegas_Grand_Prix', 'Abu_Dhabi_Grand_Prix'])

In [7]:
team_radio22.keys()

dict_keys(['Bahrain_Grand_Prix', 'Saudi_Arabian_Grand_Prix', 'Australian_Grand_Prix', 'Emilia_Romagna_Grand_Prix', 'Miami_Grand_Prix', 'Spanish_Grand_Prix', 'Monaco_Grand_Prix', 'Azerbaijan_Grand_Prix', 'Canadian_Grand_Prix', 'British_Grand_Prix', 'Austrian_Grand_Prix', 'French_Grand_Prix', 'Hungarian_Grand_Prix', 'Belgian_Grand_Prix', 'Dutch_Grand_Prix', 'Italian_Grand_Prix', 'Singapore_Grand_Prix', 'Japanese_Grand_Prix', 'United_States_Grand_Prix', 'Mexico_City_Grand_Prix', 'São_Paulo_Grand_Prix', 'Abu_Dhabi_Grand_Prix'])

Common races: 'Bahrain_Grand_Prix', 'Saudi_Arabian_Grand_Prix', 'Australian_Grand_Prix', 'Azerbaijan_Grand_Prix', 'Miami_Grand_Prix', 'Monaco_Grand_Prix', 'Spanish_Grand_Prix', 'Canadian_Grand_Prix', 'Austrian_Grand_Prix', 'British_Grand_Prix', 'Hungarian_Grand_Prix', 'Belgian_Grand_Prix', 'Dutch_Grand_Prix', 'Italian_Grand_Prix', 'Singapore_Grand_Prix', 'Japanese_Grand_Prix', 'United_States_Grand_Prix', 'Mexico_City_Grand_Prix', 'São_Paulo_Grand_Prix', 'Abu_Dhabi_Grand_Prix'

Before we continue, we need to be cautious of potential issues/missing from our data.

There might be certain races in which a driver is in the list of session drivers BUT not in the list of team radio drivers or vice-versa.

We need to detect these certain cases before we join 2 data objects (team radio timestamps & telemetry data) by driver number!

In [9]:
def analyse_season(dictt, year):
    i=1
    for k,v in dictt.items():
        session = fastf1.get_session(year, k,'Race') 
        session.load()
        
        drivers = sorted(session.drivers)
        rd = v.index.get_level_values(0).unique()
        
        different_drivers = [x for x in drivers if x not in set(rd)]
        different_drivers.append([x for x in rd if x not in set(drivers)])
        
        print(i," ",k,"\n")
        print("RACE \n",len(drivers)," ",drivers)
        print("RADIO \n",len(rd)," ",rd)
        if different_drivers !=[[]]: # if there is an inbalance between team radio drivers & session drivers in a race!
            print("\n Difference: ",different_drivers)
        
        i+=1

In [8]:
def driver_from_session(grand_prix,number):
    s= fastf1.get_session(2023, grand_prix,'Race') 
    s.load()
    print(s.get_driver(number))
    
def driver_from_radio(grand_prix,number):
    df = team_radio[grand_prix] 
    print(df.loc[(number, slice(None))]['Path New'][0])

In [ ]:
analyse_season(team_radio22, 2022)

## 2022 SEASON
### SAUDI GP
**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '3', '31', '4', '44', '47', '55', '6', '63', '77']
- 
**RADIO** 
- 19   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '3',
       '31', '4', '44', '55', '6', '63', '77']

**Difference:** ['47']
- No radio for 	Mick Schumacher(47), but he raced.

### Miami GP
**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '3', '31', '4', '44', '47', '5', '55', '6', '63', '77']
- 
**RADIO**
- 18   ['1', '10', '14', '16', '18', '20', '22', '23', '3', '31', '4', '44',
       '47', '5', '55', '6', '63', '77']
       
### British GP 
**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '3', '31', '4', '44', '47', '5', '55', '6', '63', '77']
- 
**RADIO**
- 18   ['1', '10', '11', '14', '16', '18', '20', '22', '3', '31', '4', '44',
       '47', '5', '55', '6', '63', '77']

**Difference:** ['23', '24']

### Singapore GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '3', '31', '4', '44', '47', '5', '55', '6', '63', '77']

**RADIO**
 19   ['1', '10', '11', '16', '18', '20', '22', '23', '24', '3', '31', '4',
       '44', '47', '5', '55', '6', '63', '77']

**Difference:**  ['14']

### São Paulo GP 

**RACE** 
- 20   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '3', '31', '4', '44', '47', '5', '55', '6', '63', '77']
**RADIO** 
- 19   ['1', '10', '11', '14', '16', '18', '20', '22', '23', '24', '31', '4',
       '44', '47', '5', '55', '6', '63', '77']

**Difference:**  ['3']

### GPs WHERE ALL DRIVERS MATCH!

- Bahrain GP
- Australian GP
- Emilia Romagna GP
- Spanish GP
- Monaco GP
- Azerbaijan GP
- Canadian GP
- Austrian GP
- French GP
- Hungarian GP
- Belgian GP
- Italian GP
- Dutch GP
- Japanese GP
- United States GP
- Mexico City GP
- Abu Dhabi GP

In [ ]:
analyse_season(team_radio, 2023)

## 2023 SEASON
### Canadian GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']

**Difference:** ['21']
- No radio for Nyck De Vries(21), but he raced.

### Hungarian GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 18   ['1', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']

**Difference:** ['10', '3']
- No radio for Pierre Gasly(10) & Daniel Ricciardo(3), but they raced.

### Dutch GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '31', '4', '40', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '27', '31', '4', '40', '44', '55', '63', '77', '81']

**Difference:** ['24']
- No radio for Guanyu Zhou(24), but he raced.

### Singapore GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '31', '4', '40', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '2', '20', '22', '23', '24', '27', '31', '4', '40', '44', '55', '63', '77', '81']

**Difference:** ['18']
- No radio for Lance Stroll(18), he did not race due to an injury.

### Qatar GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '31', '4', '40', '44', '63', '77', '81']

**Difference:** ['3', '55', '40']
- No radio for Lewis Hamilton(55), he DNF'ed.
- No radio for Daniel Ricciardo(3), he did not race due to an injury.

- There is radio available for Liam Lawson(40) but he is not on the driver list, he was a replacement of Ricciardo(3).
- Even though, Daniel Ricciardo(3) did not race, he is in the list of drivers.

### United States GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**Difference:** ['21', '3']
- Nyck De Vries(21) is in the driver list, even though he did not race.
- There is radio available for Daniel Ricciardo(3) but he is not on the driver list.
- Daniel Ricciardo(3) was a replacement for Nyck De Vries(21).

### São Paulo GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**Difference:** ['23']
- No radio for Alexander Albon(23), he DNF'ed.

### Las Vegas GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '44', '55', '63', '77', '81']

**Difference:** ['4']
- No radio for Lando Norris(4), he DNF'ed.

### Abu Dhabi GP

**RACE**
- 20   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**RADIO**
- 19   ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '27', '3', '31', '4', '44', '55', '63', '77', '81']

**Difference:** ['24']
- No radio for Zhou Guanyu(24), but he raced.

### GPs WHERE ALL DRIVERS MATCH!

- Bahrain GP
- Saudi GP
- Australian GP
- Azerbaijan GP
- Miami GP
- Monaco GP
- Spanish GP
- Austrian GP
- British GP
- Belgian GP
- Italian GP
- Japanese GP
- Mexico City GP


In [4]:
def match_timestamps(date, telemetry):
    
    """
    Right join function where it returns the telemetry data of timestamps
    where radio conversation occurs.
    :param date: Team radio timestamps.
    :param telemetry: All timestamps available in telemetry data
    :return: data frame containing telemetry
    """
    
    dfs = []
    team_radio_ts = date
    telemetry_ts = telemetry['Date'].values 
    
    for ts in team_radio_ts:
        index = np.where(telemetry_ts == ts)[0]
        if len(index) > 0:
            df = telemetry.iloc[[index[0]], :]
            dfs.append(df)
        else:
            index = np.where(telemetry['Date'] < ts)[0]
            if len(index) > 0:
                df = telemetry.iloc[[index[-1]], :]
                dfs.append(df)
            else: 
                index = np.where(telemetry['Date'] > ts)[0]
                df = telemetry.iloc[[index[0]], :]
                dfs.append(df)

    result = pd.concat(dfs, ignore_index=True) 
    return result

In [5]:
def match_timestamps_reverse(date, telemetry):
    
    """
    :param date: Team radio timestamps.
    :param telemetry: All timestamps available in telemetry data
    :return: Index list of telemetry data where radio conversations occur.
    """

    index_list = []
    team_radio_ts = date
    telemetry_ts = telemetry['Date'].values 
    
    for ts in team_radio_ts:
        index = np.where(telemetry_ts == ts)[0]
        if len(index) > 0:
            index_list.append(index[0])
        else:
            index = np.where(telemetry['Date'] < ts)[0]
            if len(index) > 0:
                index_list.append(index[-1])
            else: 
                index = np.where(telemetry['Date'] > ts)[0]
                index_list.append(index[0])

    return index_list

In [6]:
def dict2df(dictionary):
    dfs = []
    for racing_number, data_list in dictionary.items():
        df = pd.DataFrame(data_list)
        dfs.append(df)
    
    result = pd.concat(dfs, keys=dictionary.keys())
    return result

In [14]:
correct_gps22= grand_prix = ["Bahrain_Grand_Prix","Australian_Grand_Prix","Emilia_Romagna_Grand_Prix","Spanish_Grand_Prix","Monaco_Grand_Prix","Azerbaijan_Grand_Prix","Canadian_Grand_Prix","Austrian_Grand_Prix","French_Grand_Prix","Hungarian_Grand_Prix","Belgian_Grand_Prix","Italian_Grand_Prix","Dutch_Grand_Prix","Japanese_Grand_Prix","United_States_Grand_Prix","Mexico_City_Grand_Prix","Abu_Dhabi_Grand_Prix"]

correct_gps23 = ['Bahrain_Grand_Prix','Saudi_Arabian_Grand_Prix','Australian_Grand_Prix','Azerbaijan_Grand_Prix','Miami_Grand_Prix','Monaco_Grand_Prix','Spanish_Grand_Prix','Austrian_Grand_Prix','British_Grand_Prix',
'Belgian_Grand_Prix','Italian_Grand_Prix','Japanese_Grand_Prix','Mexico_City_Grand_Prix']

In [18]:
def create_dict(team_radio_dict,gp, year):
    f1_dict = {}
    for k,v in team_radio_dict.items():
        # k is the country
        # v is the data frame of the related country with columns: 'Utc', 'RacingNumber', 'Path', 'Path New', 'Date'
        if np.isin(gp,k).any():
            session = fastf1.get_session(year, k,'Race') 
            session.load()
            
            drivers = sorted(session.drivers)
            
            #f1[k] = {}
            f1_dict[k] = {}
            for driver in drivers:
                df = v.loc[(driver, slice(None))]
                dates = df['Date'] #timestamps where team radio conversation occurs
                
                tel = session.laps.pick_driver(driver).get_telemetry()
                tel = tel[['Date','X','Y','Z','RPM','Brake','DRS','Throttle','Speed', 'nGear']] 
                
                index_list = match_timestamps_reverse(dates, tel)
                telemetry = tel.copy()
                telemetry.reset_index()
                telemetry.insert(len(telemetry.columns),"Team Radio",0)
                
                for i in index_list:
                    telemetry.loc[i,'Team Radio']=1
                    
                telemetry = telemetry.reset_index(drop=True)
                f1_dict[k][driver] = telemetry
                
    return f1_dict

In [10]:
v=team_radio['Dutch_Grand_Prix']

In [11]:
session = fastf1.get_session(2023, 'Dutch_Grand_Prix','Race') 
session.load()
df = v.loc[('1', slice(None))]
dates = df['Date'] #timestamps where team radio conversation occurs

tel = session.laps.pick_driver('1').get_telemetry()
tel = tel[['Date','X','Y','Z','RPM','Brake','DRS','Throttle','Speed', 'nGear']] 

req         WARNING 	DEFAULT CACHE ENABLED! (4.42 GB) C:\Users\sinem\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Dutch Grand Prix - Race [v3.3.2]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '10', '11', '55', '44', '4', '23', '81', '31', '18', '27', '40', '77', '22

In [13]:
tel

,Date,X,Y,Z,RPM,Brake,DRS,Throttle,Speed,nGear
2,2023-08-27 13:03:05.334,627.102635,4068.147684,537.000000,10088,False,1,15,0,1
3,2023-08-27 13:03:05.416,627.063361,4068.091758,537.000000,10093,False,1,15,0,1
4,2023-08-27 13:03:05.475,627.000000,4068.000000,537.000000,10078,False,1,15,0,1
5,2023-08-27 13:03:05.616,626.732545,4067.612678,537.000000,10063,False,1,15,0,1
6,2023-08-27 13:03:05.895,627.000000,4069.000000,537.000000,9290,False,1,15,0,1
...,...,...,...,...,...,...,...,...,...,...
65044,2023-08-27 15:27:09.370,326.000000,3316.000000,537.000000,11325,False,1,100,290,7
65045,2023-08-27 15:27:09.447,349.933480,3378.183434,536.999857,11311,False,1,100,290,7
65046,2023-08-27 15:27:09.767,423.712192,3569.586271,536.999989,11425,False,1,100,290,7
65047,2023-08-27 15:27:09.790,429.000000,3583.000000,537.000000,11441,False,1,100,290,7


In [14]:
dates

0    2023-08-27 12:11:03.647
1    2023-08-27 12:59:13.118
2    2023-08-27 13:05:59.354
3    2023-08-27 13:22:20.078
4    2023-08-27 13:47:44.176
5    2023-08-27 13:48:53.956
6    2023-08-27 13:57:52.931
7    2023-08-27 14:20:25.016
8    2023-08-27 14:22:07.530
9    2023-08-27 14:27:13.478
10   2023-08-27 14:27:15.946
11   2023-08-27 15:12:40.023
12   2023-08-27 15:13:46.022
13   2023-08-27 15:17:10.846
14   2023-08-27 15:28:58.302
Name: Date, dtype: datetime64[ns]

In [12]:
index_list = match_timestamps_reverse(dates, tel)
telemetry = tel.copy()
telemetry.reset_index()
telemetry.insert(len(telemetry.columns),"Team Radio",0)

for i in index_list:
    telemetry.loc[i,'Team Radio']=1
    
telemetry = telemetry.reset_index(drop=True)
telemetry

,Date,X,Y,Z,RPM,Brake,DRS,Throttle,Speed,nGear,Team Radio
0,2023-08-27 13:03:05.334,627.102635,4068.147684,537.000000,10088.0,False,1.0,15.0,0.0,1.0,0.0
1,2023-08-27 13:03:05.416,627.063361,4068.091758,537.000000,10093.0,False,1.0,15.0,0.0,1.0,0.0
2,2023-08-27 13:03:05.475,627.000000,4068.000000,537.000000,10078.0,False,1.0,15.0,0.0,1.0,0.0
3,2023-08-27 13:03:05.616,626.732545,4067.612678,537.000000,10063.0,False,1.0,15.0,0.0,1.0,0.0
4,2023-08-27 13:03:05.895,627.000000,4069.000000,537.000000,9290.0,False,1.0,15.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
65043,2023-08-27 15:27:09.447,349.933480,3378.183434,536.999857,11311.0,False,1.0,100.0,290.0,7.0,0.0
65044,2023-08-27 15:27:09.767,423.712192,3569.586271,536.999989,11425.0,False,1.0,100.0,290.0,7.0,1.0
65045,2023-08-27 15:27:09.790,429.000000,3583.000000,537.000000,11441.0,False,1.0,100.0,290.0,7.0,0.0
65046,2023-08-27 15:27:10.082,501.224720,3762.024622,537.000019,11449.0,False,1.0,100.0,290.0,7.0,0.0


In [15]:
telemetry.index[telemetry['Team Radio'] == 1].tolist()


[1317,
 8671,
 20203,
 20728,
 24797,
 34966,
 35753,
 38045,
 38065,
 58530,
 59021,
 60529,
 65044,
 65047]

In [18]:
tel.iloc[1315:,:]

,Date,X,Y,Z,RPM,Brake,DRS,Throttle,Speed,nGear
1317,2023-08-27 13:05:58.817,1252.023502,1626.517718,510.112007,5833,False,1,5,66,2
1318,2023-08-27 13:05:58.975,1238.000000,1601.000000,511.000000,5947,False,1,6,66,2
1319,2023-08-27 13:05:59.097,1229.173203,1578.831505,511.377195,6062,False,1,7,67,2
1320,2023-08-27 13:05:59.297,1218.397340,1540.160026,511.710055,6027,False,1,10,68,2
1321,2023-08-27 13:05:59.354,1216.000000,1530.000000,512.000000,6186,False,1,10,68,2
...,...,...,...,...,...,...,...,...,...,...
65044,2023-08-27 15:27:09.370,326.000000,3316.000000,537.000000,11325,False,1,100,290,7
65045,2023-08-27 15:27:09.447,349.933480,3378.183434,536.999857,11311,False,1,100,290,7
65046,2023-08-27 15:27:09.767,423.712192,3569.586271,536.999989,11425,False,1,100,290,7
65047,2023-08-27 15:27:09.790,429.000000,3583.000000,537.000000,11441,False,1,100,290,7


In [17]:
telemetry.iloc[1315:,:]

,Date,X,Y,Z,RPM,Brake,DRS,Throttle,Speed,nGear,Team Radio
1315,2023-08-27 13:05:58.817,1252.023502,1626.517718,510.112007,5833.0,False,1.0,5.0,66.0,2.0,0.0
1316,2023-08-27 13:05:58.975,1238.000000,1601.000000,511.000000,5947.0,False,1.0,6.0,66.0,2.0,0.0
1317,2023-08-27 13:05:59.097,1229.173203,1578.831505,511.377195,6062.0,False,1.0,7.0,67.0,2.0,1.0
1318,2023-08-27 13:05:59.297,1218.397340,1540.160026,511.710055,6027.0,False,1.0,10.0,68.0,2.0,0.0
1319,2023-08-27 13:05:59.354,1216.000000,1530.000000,512.000000,6186.0,False,1.0,10.0,68.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
65043,2023-08-27 15:27:09.447,349.933480,3378.183434,536.999857,11311.0,False,1.0,100.0,290.0,7.0,0.0
65044,2023-08-27 15:27:09.767,423.712192,3569.586271,536.999989,11425.0,False,1.0,100.0,290.0,7.0,1.0
65045,2023-08-27 15:27:09.790,429.000000,3583.000000,537.000000,11441.0,False,1.0,100.0,290.0,7.0,0.0
65046,2023-08-27 15:27:10.082,501.224720,3762.024622,537.000019,11449.0,False,1.0,100.0,290.0,7.0,0.0


In [19]:
f1_dict22 = create_dict(team_radio22,correct_gps22, 2022)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.2]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']
core           INFO 	Loading data for Australian Grand Prix - 

In [ ]:
f1_dict23 = create_dict(team_radio,correct_gps23, 2023)

check and delete null rows!

In [20]:
def change_index(full_dict):
    """
    Changing the index of the dataframes (in the dictionary) to X-Y coordinates.
    """
    for gp,sub_dict in full_dict.items():
        for driver, df in sub_dict.items():
            df['X-Y'] = list(zip(df['X'], df['Y']))
            df.set_index('X-Y',inplace=True)

In [21]:
def organize_dict(full_dict):
    """
    Organizing the values of the dictionary; by changing the indexes, converting dictionary typed values to data frames
    and dropping null rows.
    """
    change_index(full_dict)
    new_dict = {}
    for gp, sub_dict in full_dict.items():
        df = dict2df(sub_dict)
        df = df.dropna()
        new_dict[gp] = df
    return new_dict

In [22]:
f1_22 = organize_dict(f1_dict22)
f1_23 = organize_dict(f1_dict23)

In [23]:
f1_22['Japanese_Grand_Prix']

Date  \
   X-Y                                                                 
1  (3511.9841901852997, -2961.017263356482)  2022-10-09 05:03:35.902   
   (3512.0, -2961.0)                         2022-10-09 05:03:36.020   
   (3512.0002222924504, -2960.999757257725)  2022-10-09 05:03:36.021   
   (3512.0590792070593, -2960.9354857933386) 2022-10-09 05:03:36.221   
   (3512.0, -2961.0)                         2022-10-09 05:03:36.320   
...                                                              ...   
77 (0.0, 0.0)                                2022-10-09 08:06:46.282   
   (0.0, 0.0)                                2022-10-09 08:06:46.412   
   (0.0, 0.0)                                2022-10-09 08:06:46.652   
   (0.0, 0.0)                                2022-10-09 08:06:46.661   
   (0.0, 0.0)                                2022-10-09 08:06:46.703   

                                                        X            Y  \
   X-Y                                                                   
1  (3511.9841901852997, -2961.017263356482)   3511.984190 -2961.017263   
   (3512.0, -2961.0)                          3512.000000 -2961.000000   
   (3512.0002222924504, -2960.999757257725)   3512.000222 -2960.999757   
   (3512.0590792070593, -2960.9354857933386)  3512.059079 -2960.935486   
   (3512.0, -2961.0)                          3512.000000 -2961.000000   
...                                                   ...          ...   
77 (0.0, 0.0)                                    0.000000     0.000000   
   (0.0, 0.0)                                    0.000000     0.000000   
   (0.0, 0.0)                                    0.000000     0.000000   
   (0.0, 0.0)                                    0.000000     0.000000   
   (0.0, 0.0)                                    0.000000     0.000000   

                                                       Z      RPM Brake  DRS  \
   X-Y                                                                         
1  (3511.9841901852997, -2961.017263356482)   666.000004  10168.0  True  1.0   
   (3512.0, -2961.0)                          666.000000  10188.0  True  1.0   
   (3512.0002222924504, -2960.999757257725)   666.000000  10228.0  True  1.0   
   (3512.0590792070593, -2960.9354857933386)  665.999984   9948.0  True  1.0   
   (3512.0, -2961.0)                          666.000000   8865.0  True  1.0   
...                                                  ...      ...   ...  ...   
77 (0.0, 0.0)                                   0.000000      0.0  True  9.0   
   (0.0, 0.0)                                   0.000000      0.0  True  9.0   
   (0.0, 0.0)                                   0.000000      0.0  True  9.0   
   (0.0, 0.0)                                   0.000000      0.0  True  9.0   
   (0.0, 0.0)                                   0.000000      0.0  True  9.0   

                                              Throttle  Speed  nGear  \
   X-Y                                                                 
1  (3511.9841901852997, -2961.017263356482)       11.0    0.0    2.0   
   (3512.0, -2961.0)                              11.0    0.0    2.0   
   (3512.0002222924504, -2960.999757257725)       11.0    0.0    2.0   
   (3512.0590792070593, -2960.9354857933386)      11.0    0.0    2.0   
   (3512.0, -2961.0)                              11.0    0.0    2.0   
...                                                ...    ...    ...   
77 (0.0, 0.0)                                    104.0    0.0    0.0   
   (0.0, 0.0)                                    104.0    0.0    0.0   
   (0.0, 0.0)                                    104.0    0.0    0.0   
   (0.0, 0.0)                                    104.0    0.0    0.0   
   (0.0, 0.0)                                    104.0    0.0    0.0   

                                              Team Radio  
   X-Y                                                    
1  (3511.9841901852997, -2961.017263356482)          0.0  
   (3512.

In [ ]:
f1_23['Japanese_Grand_Prix']

In [24]:
with open(os.path.join(data_path,'f1_22.pkl'), 'wb') as f:
    pickle.dump(f1_22, f)

with open(os.path.join(data_path,'f1_23.pkl'), 'wb') as f:
    pickle.dump(f1_23, f)